In [1]:
import numpy as np
import pandas as pd
import scipy as sp
from scipy.spatial.distance import cdist
from skimage import measure
from matplotlib import pyplot as plt
import os
import tifffile
from tqdm import tqdm
from typing import List, Tuple

In [2]:
selected_magnification = "20x"
selected_image_type = "raw"
max_dist = 9
aspect = np.array((1/3, 1/3))    # XY: 20x / aspect => 60x

### Read shift data

In [3]:
shifts = pd.read_csv(f"shift_correction/{selected_magnification}_{selected_image_type}.shifts.csv")
shifts.index = shifts["sid"].values
shifts.drop("sid", 1, inplace=True)

# Matching 20x_dw and reference dots

In [4]:
dots_data = pd.read_csv("/mnt/data/Imaging/202105-Deconwolf/data_210726/dots_data.clean.tsv.gz", sep="\t")
dots_data = dots_data[selected_magnification == dots_data["magnification"]]
dots_data = dots_data[selected_image_type == dots_data["image_type"]]

In [5]:
thresholds_table = pd.read_csv("../../data/magnifications_matching/intensity_thresholds.by_field.tsv", sep="\t")

In [6]:
reference = pd.read_csv("../../data/60x_reference/ref__dw.field_thr.tsv", sep="\t")

In [7]:
def get_unassigned_count(pdist: np.ndarray) -> int:
    return np.sum(np.nansum(np.isnan(pdist), 1) != pdist.shape[1])

def match_dots(dotsA: np.ndarray, dotsB: np.ndarray) -> List[Tuple[int, int, float]]:
    pdist = cdist(dotsA, dotsB)
    
    matched: List[Tuple[int, int, float]] = []
    unassigned_rows = pdist.shape[0] + 1
    iteration_counter = 0
    print("Iteration #", end="")
    while unassigned_rows > get_unassigned_count(pdist) and 0 < get_unassigned_count(pdist):
        iteration_counter += 1
        print(f"...{iteration_counter}", end="")

        unassigned_rows = get_unassigned_count(pdist)
        A_id = 0
        while A_id < pdist.shape[0]:
            if pdist.shape[1] == np.sum(np.isnan(pdist[A_id, :])):
                A_id += 1
                continue
            B_id = np.nanargmin(pdist[A_id, :])
            if np.nanargmin(pdist[:, B_id]) == A_id:
                matched.append((A_id, B_id, pdist[A_id, B_id]))
                pdist[A_id, :] = np.nan
                pdist[:, B_id] = np.nan
            A_id += 1
    print("")
    return matched

In [8]:
match_output: List[pd.DataFrame] = []
match_output_full: List[pd.DataFrame] = []
match_counts: List[Tuple[int, int, int, int]] = []
for current_field_id in tqdm(np.unique(dots_data["sid"])):
    thresholds = thresholds_table.loc[current_field_id == thresholds_table["sid"], :]
    intensity_thr = thresholds.loc[selected_image_type == thresholds["image_type"], "thr"].values[0]

    dots = dots_data.loc[current_field_id == dots_data["sid"], :].copy(
        ).sort_values("Value2", ascending=False).reset_index(drop=True)
    
    dot_coords = dots.loc[intensity_thr <= dots["Value2"], ("x", "y")].copy().reset_index(drop=True)
    dot_coords2 = dot_coords.copy() / aspect
    dot_coords2["x"] += (shifts.loc[current_field_id, "x"] * 9)
    dot_coords2["y"] += (shifts.loc[current_field_id, "y"] * 9)
    ref_coords = reference.loc[reference["sid"] == current_field_id, ("x", "y")].copy().reset_index(drop=True)
    
    matched: List[Tuple[int, int, float]] = match_dots(dot_coords2, ref_coords)
    
    matched_a = np.array(matched)
    matched_field = pd.DataFrame(dict(
        series=current_field_id,
        id_20x=matched_a[:, 0].astype("i"),
        id_60x=matched_a[:, 1].astype("i"),
        eudist=matched_a[:, 2]
    ))
    matched_field_within_dist = matched_field[matched_field["eudist"] <= max_dist]
    match_output.append(matched_field_within_dist)
    match_output_full.append(matched_field)
    match_counts.append((current_field_id,
                         matched_field_within_dist.shape[0],
                         matched_field.shape[0]-matched_field_within_dist.shape[0],
                         dot_coords2.shape[0],
                         ref_coords.shape[0]))

  0%|          | 0/7 [00:00<?, ?it/s]

Iteration #...1...2...3...4...5

 14%|█▍        | 1/7 [00:02<00:14,  2.49s/it]


Iteration #...1...2...3...4...5...6...7...8

 29%|██▊       | 2/7 [00:08<00:17,  3.56s/it]

...


Iteration #...1...2...3...4...5...6...7...8...9...10...11...12...13...14...15...16...17...18...19...20...21...22...23...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...40...41...42...43...44...45...46...47...48...49

 86%|████████▌ | 6/7 [00:35<00:06,  6.15s/it]


Iteration #...1...2...3...4...5...6...7...8...9...10...11...12...13...14

100%|██████████| 7/7 [00:36<00:00,  5.19s/it]

...15


In [9]:
match_counts_a = np.array(match_counts)
n_total_matched = np.sum(match_counts_a[:, 1])
n_total_over_dist = np.sum(match_counts_a[:, 2])
n_total_20x = np.sum(match_counts_a[:, 3])
n_total_60x = np.sum(match_counts_a[:, 4])
print(f"{n_total_matched}/{n_total_20x} ({n_total_matched/n_total_20x*100:.2f}%) 20x dots matched to 60x")
print(f"{n_total_matched}/{n_total_60x} ({n_total_matched/n_total_60x*100:.2f}%) 60x dots matched to 20x")
print(f"{n_total_over_dist} ({n_total_over_dist/n_total_20x*100:.2f}% 20x, {n_total_over_dist/n_total_60x*100:.2f}% 60x) dots matched with distance > {max_dist}")

6433/15385 (41.81%) 20x dots matched to 60x
6433/9948 (64.67%) 60x dots matched to 20x
3048 (19.81% 20x, 30.64% 60x) dots matched with distance > 9


In [10]:
for series_id in range(match_counts_a.shape[0]):
    n_matched = match_counts_a[series_id, 1]
    n_over_dist = match_counts_a[series_id, 2]
    n_20x = match_counts_a[series_id, 3]
    n_60x = match_counts_a[series_id, 4]
    print(f"""Field #{series_id+1}
    {n_matched}/{n_20x} ({n_matched/n_20x*100:.2f}%) 20x dots matched to 60x
    {n_matched}/{n_60x} ({n_matched/n_60x*100:.2f}%) 60x dots matched to 20x
    {n_over_dist} ({n_over_dist/n_20x*100:.2f}% 20x, {n_over_dist/n_60x*100:.2f}% 60x) dots matched with distance > {max_dist}
    """)

Field #1
    1038/2203 (47.12%) 20x dots matched to 60x
    1038/1272 (81.60%) 60x dots matched to 20x
    234 (10.62% 20x, 18.40% 60x) dots matched with distance > 9
    
Field #2
    1487/3098 (48.00%) 20x dots matched to 60x
    1487/1865 (79.73%) 60x dots matched to 20x
    378 (12.20% 20x, 20.27% 60x) dots matched with distance > 9
    
Field #3
    734/1784 (41.14%) 20x dots matched to 60x
    734/1032 (71.12%) 60x dots matched to 20x
    298 (16.70% 20x, 28.88% 60x) dots matched with distance > 9
    
Field #4
    1318/3009 (43.80%) 20x dots matched to 60x
    1318/1534 (85.92%) 60x dots matched to 20x
    216 (7.18% 20x, 14.08% 60x) dots matched with distance > 9
    
Field #5
    1226/3814 (32.14%) 20x dots matched to 60x
    1226/2471 (49.62%) 60x dots matched to 20x
    1245 (32.64% 20x, 50.38% 60x) dots matched with distance > 9
    
Field #6
    501/1040 (48.17%) 20x dots matched to 60x
    501/870 (57.59%) 60x dots matched to 20x
    369 (35.48% 20x, 42.41% 60x) dots matc

#### General statistics, excluding field #7

In [11]:
match_counts_a = np.array(match_counts)
n_total_matched = np.sum(match_counts_a[:6, 1])
n_total_over_dist = np.sum(match_counts_a[:6, 2])
n_total_20x = np.sum(match_counts_a[:6, 3])
n_total_60x = np.sum(match_counts_a[:6, 4])
print(f"{n_total_matched}/{n_total_20x} ({n_total_matched/n_total_20x*100:.2f}%) 20x dots matched to 60x")
print(f"{n_total_matched}/{n_total_60x} ({n_total_matched/n_total_60x*100:.2f}%) 60x dots matched to 20x")
print(f"{n_total_over_dist} ({n_total_over_dist/n_total_20x*100:.2f}% 20x, {n_total_over_dist/n_total_60x*100:.2f}% 60x) dots matched with distance > {max_dist}")

6304/14948 (42.17%) 20x dots matched to 60x
6304/9044 (69.70%) 60x dots matched to 20x
2740 (18.33% 20x, 30.30% 60x) dots matched with distance > 9


#### General statistics, excluding fields #6-7

In [12]:
match_counts_a = np.array(match_counts)
n_total_matched = np.sum(match_counts_a[:5, 1])
n_total_over_dist = np.sum(match_counts_a[:5, 2])
n_total_20x = np.sum(match_counts_a[:5, 3])
n_total_60x = np.sum(match_counts_a[:5, 4])
print(f"{n_total_matched}/{n_total_20x} ({n_total_matched/n_total_20x*100:.2f}%) 20x dots matched to 60x")
print(f"{n_total_matched}/{n_total_60x} ({n_total_matched/n_total_60x*100:.2f}%) 60x dots matched to 20x")
print(f"{n_total_over_dist} ({n_total_over_dist/n_total_20x*100:.2f}% 20x, {n_total_over_dist/n_total_60x*100:.2f}% 60x) dots matched with distance > {max_dist}")

5803/13908 (41.72%) 20x dots matched to 60x
5803/8174 (70.99%) 60x dots matched to 20x
2371 (17.05% 20x, 29.01% 60x) dots matched with distance > 9


### Export

In [13]:
matched_dots = pd.concat(match_output)

In [14]:
matched_dots.to_csv(os.path.join("../../data/magnifications_matching",
                                 f"{selected_magnification}_{selected_image_type}.matched_dots.field_thr.tsv"
                                ), sep="\t", index=False)

In [15]:
pd.DataFrame(match_counts_a, columns=["series_id", "matched", "matched_over_dist", "raw_total", "dw_total"]
            ).to_csv(os.path.join("../../data/magnifications_matching",
                                  f"{selected_magnification}_{selected_image_type}.matched_dots.field_thr.counts.tsv"
                                 ), sep="\t", index=False)